In [1]:
%matplotlib inline
import os, sys, glob, scipy, warnings
import scipy.stats as ss
from scipy.stats import pearsonr,spearmanr
from scipy.spatial.distance import squareform
import matplotlib, scipy
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.data import Brain_Data, Adjacency
from nltools.stats import one_sample_permutation, two_sample_permutation, correlation_permutation
from nltools.stats import threshold
from nltools.mask import expand_mask, collapse_mask
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

/Users/orlacamus/opt/anaconda3/envs/fMRI/lib/python3.9/site-packages/nilearn/experimental/__init__.py:10: UserWarning: 


All features in the nilearn.experimental module are experimental and subject to change. They are included in the nilearn package to gather early feedback from users about prototypes of new features. Changes may break backwards compatibility without prior notice or a deprecation cycle. Moreover, some features may be incomplete or may have been tested less thoroughly than the rest of the library.


  warnings.warn(


In [11]:
base_dir="/home/haiyanwu/nas_data/experiment_data/RDonly240615/"
glm_dir=base_dir+'1.GLM/results/GLM1_spm_unsmooth/'
rsa_dir=base_dir+'5.ISRSA/'
roi_dir=base_dir+'3.ROIextraction/'
mask_dir=roi_dir+'masks/'

In [12]:
subList=[104,105,106,108,109,110,111,112,113,114,115,117,118,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140]

In [13]:
def ensure_dir(path):
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)

In [14]:
# Fixed variables
metric_brain = 'correlation'
metric_model = 'euclidean'
permutation_method = 'vector'
n_permute = 10000
nparcel = 200
parcels = range(nparcel)
subNums=subList
subIndices=np.where(pd.DataFrame(subList).isin(subNums))[0]



In [15]:
behavDists=pd.read_csv(rsa_dir+'RD_ddmparams.csv',index_col=0)
modelDists=scipy.spatial.distance.pdist(behavDists.loc[:,['v_former_diff','v_diff']],metric=metric_model)


In [16]:
#!!!
results_dir=rsa_dir+'ISRSAresults/'
ensure_dir(results_dir)
brain_dir=base_dir+'5.ISRSA/BrainDist/'

In [2]:
ROIs=['BA8-preSMA','BA9-dlPFC','lTPJ','ACC']
len(ROIs)

4

In [29]:


pathCur = os.path.join(results_dir,'IS-RSA_ROI_perm-%s/'%(permutation_method))#!!!!
ensure_dir(pathCur)
# Brain Data
BrainDist = pd.read_csv(os.path.join(brain_dir,'BrainDist_%s.csv' %(metric_brain)))

for parcel,name in enumerate(ROIs):
    print(parcel)
    brainDist = BrainDist.iloc[:,parcel] 
    brainDist_mat = pd.DataFrame(scipy.spatial.distance.squareform(brainDist)).iloc[subIndices,subIndices]
    brainDist_vec = brainDist_mat.values[np.triu_indices(len(subIndices),k=1)] 

    # Operations
    corr = spearmanr(modelDists,brainDist_vec)[0]
    if n_permute is None:
        pval = spearmanr(modelDists,brainDist_vec)[1]
    elif isinstance(n_permute, int):
        perm = []
        if permutation_method == 'vector':
            for p in range(n_permute):
                perm.append(spearmanr(np.random.permutation(modelDists),brainDist_vec)[0])
        if corr>=0:
            perm_p = np.mean(perm>=corr)
        else:
            perm_p = np.mean(perm<=corr)
        pval = perm_p
    out = pd.DataFrame([[corr,pval]],columns=['r','p']) # one row one parcel
    out.to_csv(os.path.join(pathCur,'parcel%03d.csv'%(parcel)))
    pd.DataFrame(perm,columns=['permutation']).to_csv(os.path.join(pathCur,'perm_parcel%03d.csv'%(parcel)))
    #~1.5min


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
